In [1]:
pip install langchain pinecone-client pyllamacpp chromadb

Note: you may need to restart the kernel to use updated packages.


## Install and Import Required Modules

In [2]:
import os
os.environ["OPENAI_API_MODEL"] = "llama"
# os.environ["PINECONE_API_KEY"] = "a34bfd48-c239-41fc-a232-000531b93b0b"
# os.environ["PINECONE_ENVIRONMENT"] = "northamerica-northeast1-gcp"
os.environ["TABLE_NAME"] = "bb-table"
os.environ["INITIAL_TASK"] = "Make the list of tasks"

from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, PromptTemplate, PromptTemplate
from langchain.embeddings import LlamaCppEmbeddings
from langchain.llms import BaseLLM

from langchain.llms import GPT4All
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores.base import VectorStore
from langchain.vectorstores import Chroma
# from langchain.vectorstores import Pinecone
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from chromadb import errors as chromadb_errors

# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
# PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT", "")
YOUR_TABLE_NAME = os.getenv("TABLE_NAME", "")
INITIAL_TASK = os.getenv("INITIAL_TASK", os.getenv("FIRST_TASK", ""))

# import pinecone
# pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

# pinecone.whoami()
# Create Pinecone index
table_name = YOUR_TABLE_NAME
# dimension = 2048
# dimension = 1536
# metric = "cosine"
# pod_type = "p1"
# if table_name not in pinecone.list_indexes():
    # pinecone.create_index(
        # table_name, dimension=dimension, metric=metric, pod_type=pod_type
    # )

# Connect to the index
# index = pinecone.Index(table_name)

# index.describe_index_stats()

# Task list
# task_list = deque([])

In [ ]:
# old test with vectorstore FAISS
# ---------------------------------------------------------------- 

# from langchain.vectorstores import FAISS
# from langchain.docstore import InMemoryDocstore

In [3]:
# old test with OpenAI embeddings
# embeddings_model = OpenAIEmbeddings()
# ----------------------------------------------------------------

# Define your embedding model:
# use your own llama model location
# i did use dalai to get my .bin
# https://cocktailpeanut.github.io/dalai/#/?id=mac
# like this: 
# npx dalai llama install 7B

# trying gpt4all
import requests

from pathlib import Path
from tqdm import tqdm
local_path = './gpt4all-lora-quantized-OSX-m1.bin'  # replace with your desired local file path
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# Example model. Check https://github.com/nomic-ai/pyllamacpp for the latest models.
url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-OSX-m1.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)
# --

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callback_manager=callback_manager, verbose=True)
embeddings_model = LlamaCppEmbeddings(model_path=local_path)
# llm = LlamaCpp(model_path="../dalai/alpaca/models/7B/ggml-model-q4_0.bin")



1it [00:00, 437.09it/s]
llama_model_load: loading model from './gpt4all-lora-quantized-OSX-m1.bin' - please wait ...
./gpt4all-lora-quantized-OSX-m1.bin: invalid model file (bad magic [got 0x6d74683c want 0x67676a74])
	you most likely need to regenerate your ggml files
	the benefit is you'll get 10-100x faster load times
	see https://github.com/ggerganov/llama.cpp/issues/91
	use convert-pth-to-ggml.py to regenerate from original pth
	use migrate-ggml-2023-03-30-pr613.py if you deleted originals
llama_init_from_file: failed to load model
llama.cpp: loading model from ./gpt4all-lora-quantized-OSX-m1.bin
error loading model: unknown (magic, version) combination: 6d74683c, 0a0d3e6c; is this really a GGML file?
AVX = 0 | AVX2 = 0 | AVX512 = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 
llama_init_from_file: failed to load model


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks

In [4]:
class TaskCreationChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_creation_template = (
            "You are an task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=["result", "task_description", "incomplete_tasks", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [5]:
class TaskPrioritizationChain(LLMChain):
    """Chain to prioritize tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_prioritization_template = (
            "You are an task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)


In [6]:
class ExecutionChain(LLMChain):
    """Chain to execute tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        execution_template = (
            "You are an AI who performs one task based on the following objective: {objective}."
            " Take into account these previously completed tasks: {context}."
            " Your task: {task}."
            " Response:"
        )
        prompt = PromptTemplate(
            template=execution_template,
            input_variables=["objective", "context", "task"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)


In [7]:
def get_next_task(task_creation_chain: LLMChain, result: Dict, task_description: str, task_list: List[str], objective: str) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)
    response = task_creation_chain.run(result=result, task_description=task_description, incomplete_tasks=incomplete_tasks, objective=objective)
    new_tasks = response.split('\n')
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

### Define the BabyAGI Controller

BabyAGI composes the chains defined above in a (potentially-)infinite loop.

In [8]:
def prioritize_tasks(task_prioritization_chain: LLMChain, this_task_id: int, task_list: List[Dict], objective: str) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(task_names=task_names, next_task_id=next_task_id, objective=objective)
    new_tasks = response.split('\n')
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [9]:
def _get_top_tasks(vectorstore: Chroma, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""

    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata['task']) for item in sorted_results]

def execute_task(vectorstore: Chroma, execution_chain: LLMChain, objective: str, task: str, k: int = 5) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [11]:

class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""

    task_list: deque = Field(default_factory=deque)

    task_creation_chain: TaskCreationChain = Field(...)

    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    
    execution_chain: ExecutionChain = Field(...)

    task_id_counter: int = Field(1)

    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None
        
    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)
        
    @property
    def input_keys(self) -> List[str]:
        return ["objective"]
    
    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs['objective']
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain, result, task["task_name"], [t["task_name"] for t in self.task_list], objective
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain, this_task_id, list(self.task_list), objective
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print("\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m")
                break
        return {}

    @classmethod
    def from_llm(
        cls,
        llm: BaseLLM,
        vectorstore,
        verbose: bool = False,
        **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(
            llm, verbose=verbose
        )
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        execution_chain = ExecutionChain.from_llm(llm, verbose=verbose)
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=execution_chain,
            vectorstore=vectorstore,
            **kwargs
        )

## Connect to the Vector Store

In [13]:

# Initialize the vectorstore as empty
# import faiss
# embedding_size = 1536
# index = faiss.IndexFlatL2(embedding_size)
# vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
vectorstore = Chroma(table_name, embeddings_model, 'chromadb')

# Ensuring the vectorstore is persisting to the chromadb:
vectorstore.persist()

# llm = OpenAI(temperature=0)
# Logging of LLMChains
verbose=False
# If None, will keep on going forever
max_iterations: Optional[int] = 10
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=verbose,
    max_iterations=max_iterations
)

[2023-04-14 16:20:40,852] {posthog.py:15} INFO - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
[2023-04-14 16:20:40,853] {__init__.py:80} INFO - Running Chroma using direct local API.
[2023-04-14 16:20:40,853] {__init__.py:41} WARNING - Using embedded DuckDB with persistence: data will be stored in: chromadb
[2023-04-14 16:20:40,860] {duckdb.py:430} INFO - loaded in 0 embeddings
[2023-04-14 16:20:40,862] {duckdb.py:440} INFO - loaded in 1 collections
[2023-04-14 16:20:40,862] {duckdb.py:85} INFO - collection with name bb-table already exists, returning existing collection
[2023-04-14 16:20:40,863] {duckdb.py:388} INFO - Persisting DB to disk, putting it in the save folder: chromadb


ValidationError: 1 validation error for BabyAGI
vectorstore
  instance of UndefinedType expected (type=type_error.arbitrary_type; expected_arbitrary_type=UndefinedType)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
baby_agi({"objective":  "say hello world"})